In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook("lab09-sp24.ipynb")

# Lab 9: Regression

Welcome to Lab 9!

Today we will get some hands-on practice with linear regression. You can find more information about this topic in
[Chapter 15.2](https://www.inferentialthinking.com/chapters/15/2/Regression_Line.html#the-regression-line).

**Submission**: Once you’re finished, run all cells besides the last one, select File > Save Notebook, and then execute the final cell. Then submit the downloaded zip file, that includes your notebook,  according to your instructor's directions.

In [ ]:
# Run this cell, but please don't change it.

# These lines import the Numpy and Datascience modules.
import numpy as np
from datascience import *

# These lines do some fancy plotting magic.
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')
import warnings
warnings.simplefilter('ignore', FutureWarning)

import d8error

# 1. How old is that abalone? 

Abalone are shellfish, specifically a type of large sea mollusk that thrives in cold waters worldwide. Their shells have a lustrous substance known as mother-of-pearl, making them sought after for jewelry, art, and decoration. Run the cell below to learn about abalone.

In [ ]:
# For the curious: this is how to display a YouTube video in a
# Jupyter notebook.  The argument to YouTubeVideo is the part
# of the URL (called a "query parameter") that identifies the
# video.  For example, the full URL for this video is:
#   https://www.youtube.com/watch?v=A2M1yKm_x1M
from IPython.display import YouTubeVideo
YouTubeVideo("A2M1yKm_x1M")

How old is an abalone? Rings form within the inner shell of the abalone, with one ring typically developing each year. Determining the age of an abalone involves cutting the shell through the cone, staining it, and counting the rings under a microscope — a tedious and time-consuming process. However, alternative measurements exist that are easier to obtain and can be used to predict the age of the abalone.
 
Today, we will use a dataset with abalone measurements and ages to see if we can make such predictions accurately with linear regression.

The dataset has one row for each observed abalone.  It includes the following columns:
- `Length`: longest shell measurement of the abalone in millimeters (mm) 
- `Shell weight`: shell weight of the abalone after being dried in grams
- `Rings`: number of rings on the abalone shell. Note: the number of rings corresponds to the age of the abalone.

Run the next cell to load the dataset.

In [ ]:
abalone = Table.read_table("abaloneI.csv")
abalone.show(10)

**Question 1.0.** The following statements are the unordered steps of linear regression.  

1. Use the regression line to generate predictions for each x value. 
2. Visualize the data and calculate the correlation coefficient to assess the whether linear regression is appropriate for modeling the data.
3. Calculate the regression line's parameters, including the slope and intercept.
4. Analyze the effectiveness of the regression line by computing the RMSE and examining the residuals plot.

Make an array called `least_squares_order` that contains the correct order of a linear regression analysis, where the first item of the array is the first step of an linear regression analysis and the last item of the array is the last step of an linear regression analysis.


In [ ]:
least_squares_order = ...

In [ ]:
grader.check("q1_0")

We would like to use linear regression to make predictions, but that won't work well if the data aren't roughly linearly related.  To check that, we should look at the data.

**Question 1.1.** Make a scatter plot of the data using shell weight to predict the number of rings on the abalone shell (ie., the age of the abalone).  It's conventional to put the column we want to predict on the vertical axis and the other column on the horizontal axis.


In [ ]:
...

**Question 1.2.** Are shell weight and rings (i.e., age) of the abalone roughly linearly related based on the scatter plot above? Is this relationship positive? **(2 points)**


_Type your answer here, replacing this text._

We're going to continue with the assumption that they are linearly related, so it's reasonable to use linear regression to analyze this data.

We'd next like to plot the data in standard units. If you don't remember the definition of standard units, textbook section [14.2](https://www.inferentialthinking.com/chapters/14/2/Variability.html#standard-units) might help!

**Question 1.3.** Compute the mean and standard deviation of the shell lengths and rings.  **Then** create a table called `abalone_standard` containing the shell lengths and rings in standard units.  The columns should be named `Shell weight (standard units)` and `Rings (standard units)`.


In [ ]:
weight_mean = ...
weight_std = ...
rings_mean = ...
rings_std = ...

abalone_standard = Table().with_columns(
    "Shell weight (standard units)", ...,
    "Rings (standard units)", ...)
abalone_standard

In [ ]:
grader.check("q1_3")

**Question 1.4.** Plot the data again, but this time in standard units.


In [ ]:
...

You'll notice that this plot looks the same as the last one!  However, the data and axes are scaled differently.  So it's important to read the ticks on the axes.

**Question 1.5.** Among the following numbers, which would you guess is closest to the correlation between shell weight and number of rings in this dataset?

1. 1
2. 0
3. -1

Assign `correlation` to the number corresponding to your guess (either 1, 2 or 3).


In [ ]:
correlation = ...

In [ ]:
grader.check("q1_5")

**Question 1.6.** Compute the correlation coefficient: `r`.  

*Hint:* Use `abalone_standard`.  Section [15.1](https://www.inferentialthinking.com/chapters/15/1/Correlation.html#calculating-r) explains how to do this.



In [ ]:
r = ...
r

In [ ]:
grader.check("q1_6")

## 2. The regression line
Recall that the **correlation** is the **slope of the regression line when the data are put in standard units**.

The next cell plots the regression line in standard units:

$$\text{number of rings (i.e., age) in standard units} = r \times \text{shell weight in standard units}$$

Then, it plots the data in standard units again, for comparison.

In [ ]:
def plot_data_and_line(dataset, x, y, point_0, point_1):
    """Makes a scatter plot of the dataset, along with a line passing through two points."""
    dataset.scatter(x, y, label="data")
    xs, ys = zip(point_0, point_1)
    plots.plot(xs, ys, label="regression line")
    plots.legend(bbox_to_anchor=(1.5,.8))

plot_data_and_line(abalone_standard, 
                   "Shell weight (standard units)", 
                   "Rings (standard units)", 
                   [-2, -2*r], 
                   [4, 4*r])

How would you take a point in standard units and convert it back to original units?  We'd have to "stretch" its horizontal position by `weight_std` and its vertical position by `rings_std`. That means the same thing would happen to the slope of the line.

Stretching a line horizontally makes it less steep, so we divide the slope by the stretching factor.  Stretching a line vertically makes it more steep, so we multiply the slope by the stretching factor.

**Question 2.1.** Calculate the slope of the regression line in original units, and assign it to `slope`.

(If the "stretching" explanation is unintuitive, consult section [15.2](https://www.inferentialthinking.com/chapters/15/2/Regression_Line.html#the-equation-of-the-regression-line) in the textbook.)


In [ ]:
slope = ...
slope

In [ ]:
grader.check("q2_1")

We know that the regression line passes through the point `(weight_mean, rings_mean)`. Recall that the equation of the regression line in the original units is:

$$\text{rings} = \text{slope} \times \text{weight} + (- \text{slope} \times \text{weight\_mean + rings\_mean})$$


**Question 2.2.** Calculate the intercept in original units and assign it to `intercept`. [Section 15.2.5](https://inferentialthinking.com/chapters/15/2/Regression_Line.html#the-regression-line-in-standard-units) may be helpful.


In [ ]:
intercept = ...
intercept

In [ ]:
grader.check("q2_2")

## 3. Investigating the regression line
The slope and intercept tell you exactly what the regression line looks like.  To predict the age of an abalone, multiply the abalone's shell weight by `slope` and then add `intercept`.

**Question 3.1.** Compute the predicted number of rings (i.e., age) for an abalone that has a shell weight of .01 grams, and for an abalone that has a shell weight of .4 grams.


In [ ]:
point_01_predicted_age = ...
point_4_predicted_age = ...

# Here is a helper function to print out your predictions.
# Don't modify the code below.
def print_prediction(weight, predicted_age):
    print("With a shell weight of", weight,
          "grams, we predict the abalone is", predicted_age,
          "years old.")

print_prediction(.01, point_01_predicted_age)
print_prediction(.4, point_4_predicted_age)

In [ ]:
grader.check("q3_1")

The next cell plots the line that goes between those two points, which is (a segment of) the regression line.

In [ ]:
plot_data_and_line(abalone, "Shell weight", "Rings", 
                   [.01, point_01_predicted_age], 
                   [.4, point_4_predicted_age])

**Question 3.2.** Make predictions for the number of rings for each abalone in the `abalone` table.  (Of course, we know exactly what the number of rings are for each abalone!  We are doing this so we can see how accurate our predictions are.  Put these numbers into a column in a new table called `abalone_predictions`.  Its first row should look like this:

|Length|Shell weight|Rings|predicted Rings|
|-|-|-|-|
|.33|.055|7|6.28421|

*Hint:* Your answer can be just one line, though you are not limited to one line.  There is no need for a `for` loop; use array arithmetic instead.


In [ ]:
abalone_predictions = ...
abalone_predictions

In [ ]:
grader.check("q3_2")

**Question 3.3.** How close were we?  Compute the *residual* for each eruption in the dataset.  The residual is the actual number of rings time minus the predicted number of rings.  Add the residuals to `abalone_predictions` as a new column called `residual` and name the resulting table `abalone_residuals`.

*Hint:* Again, your code will be much simpler if you don't use a `for` loop.


In [ ]:
abalone_residuals = ...
abalone_residuals

In [ ]:
grader.check("q3_3")

Here is a plot of the residuals you computed.  Each point corresponds to one abalone.  It shows how much our prediction over- or under-estimated the age of the abalone.

In [ ]:
abalone_residuals.scatter("Shell weight", "residual", color="r")

There isn't really a pattern in the residuals, which confirms that it was reasonable to try linear regression. 

## 4. How accurate are different predictions?
Earlier, you should have found that the correlation is moderately close to 1, so the line fits moderately well on the training data.  That means the residuals can vary widely (not all points are close to 0) in comparison to the number of rings (ie., age) of the abalone.

We can see that visually by plotting the shell weight and residuals together:

In [ ]:
# Just run this cell.
abalone_residuals.scatter("Shell weight", "Rings", label="actual number of rings", color="blue")
plots.scatter(abalone_residuals.column("Shell weight"), abalone_residuals.column("residual"), label="residual", color="r")
plots.plot([0, .5], [point_1_predicted_age, point_3_predicted_age], label="regression line")
plots.legend(bbox_to_anchor=(.2,.5));

Unless you have a strong reason to believe that the linear regression model is true, you should be wary of applying your prediction model to data that are very different from the training data.

**Question 4.1.** In the `abalone` dataset, there is no abalone with a shell weight of exactly 0, .244, or 2 grams.  Using this line, what is the predicted age for an abalone with a shell weight of 0 grams?  .244 grams?  2 grams?


In [ ]:
zero_grams_predicted_age = ...
point_248_grams_predicted_age = ...
two_grams_predicted_age = ...

print_prediction(0, zero_grams_predicted_age)
print_prediction(.248, point_248_grams_predicted_age)
print_prediction(2, two_grams_predicted_age)

In [ ]:
grader.check("q4_1")

**Question 4.2.** For each prediction, state whether you think it's reliable and explain your reasoning. **(2 points)**


_Type your answer here, replacing this text._

## 5. Checking other measurements

In addition to the abalone's shell weight, other measurements are also taken. The `abalone` dataset also includes the length of the longest shell measurement (in millimeters). 

In [ ]:
abalone.scatter("Length", "Rings")

The `standardize` function from lecture appears below, which takes in a table with numerical columns and returns the same table with each column converted into standard units.

In [ ]:
# Just run this cell.

def standard_units(any_numbers):
    "Convert any array of numbers to standard units."
    return (any_numbers - np.mean(any_numbers)) / np.std(any_numbers)  

def standardize(t):
    """Return a table in which all columns of t are converted to standard units."""
    t_su = Table()
    for label in t.labels:
        t_su = t_su.with_column(label + ' (su)', standard_units(t.column(label)))
    return t_su

**Question 5.1.** Compute the correlation coefficient *r* using the length of the abalone as the explanatory variable. 


In [ ]:
def corr_coeff(t):
    """Return the regression coefficient for columns 0 & 1."""
    t_su = standardize(t)
    ...

abalone_length = ...
abalone_length_r = ...
print("Using the length of the abalone shell, r is", abalone_length_r)

In [ ]:
grader.check("q5_1")

**Question 5.2.** Complete the functions `slope_of` and `intercept_of` below. 

When you're done, the function `age_abalone_length` should use a different regression line than the one above to predict the age of an abalone based on its length.


In [ ]:
def slope_of(table, r):
    """Return the slope of the regression line for table in original units.
    
    Assume that column 0 contains x values and column 1 contains y values.
    r is the regression coefficient for x and y.
    """
    ...

def intercept_of(table, r):
    """Return the intercept of the regression line for table in original units."""
    slope = slope_of(table, r)
    ...
    
abalone_length_slope = slope_of(abalone_length, abalone_length_r)
abalone_length_intercept = intercept_of(abalone_length, abalone_length_r)

def age_abalone_length(length):
    return abalone_length_slope * length + abalone_length_intercept

In [ ]:
grader.check("q5_2")

The plot below shows the scatter plot of length and number of rings (i.e., age) with the new regression line.

In [ ]:
abalone.scatter("Length", 'Rings', fit_line=True)

**Question 5.3.** The first row of the table produced in Question 3.2, shows an abalone with a shell weight of 0.055 grams and a length of 0.33mm. Based on the shell weight of 0.055 grams, the predicted number of rings (i.e., age of the abalone) was 6.28421. Now, use the function `age_abalone_length` from above to predict the number of rings using the shell's length, which is 0.33 mm.

In [ ]:
predicted_age_using_length =
    ...

# Don't modify the code below.
print("Using a length of 0.33 mm for the abalone, the abalone's age is", predicted_age_using_length)

In [ ]:
grader.check("q5_3")

**Question 5.4.** Do you think the predictions produced using the length of the abalone would be more accurate, less accurate, or about the same as the predictions from the regression line you created in section 2? How could you tell?

**SOLUTION:** About the same. The pattern of the data and the r value are about the same. 

## 6. Submission

<img src="lab09_pets.jpg" alt="drawing" width="500"/>

Congratulations, you're done with Lab 9!

**Important submission steps:** 
1. Run the tests and verify that they all pass.
2. Choose **Save Notebook** from the **File** menu, then **run the final cell**. 
3. Click the link to download the zip file.
4. Then submit the zip file to the corresponding assignment according to your instructor's directions. 

**It is your responsibility to make sure your work is saved before running the last cell.**

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

In [ ]:
# Save your notebook first, then run this cell to export your submission.
grader.export(pdf=False, run_tests=True)